## ImagePoemMatch

- input: 이미지키워드 토크나이징 데이터 (json) + 시 토크나이징 데이터 (json)
- output: 이미지 키워드와 시 키워드의 코사인 유사도가 가장 높은 이미지와 시 매칭 (json)

### 1. 필요한 라이브러리 import

In [1]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import json
import fasttext
from numpy import dot
from numpy.linalg import norm

### 2. 필요한 데이터 load

In [2]:
# 이미지 토픽 모음
# 'restaurant', 'tower'
image_topic = ['beach', 'cave', 'island', 'lake', 'mountain', 'amusement', 'palace', 'park']

In [3]:
# 이미지 키워드 토크나이징 데이터 저장 리스트
image_tokens = []

In [4]:
# 이미지 키워드 토크나이징 데이터 로드
for topic in image_topic:
    with open('../data/{topic}_token.json'.format(topic=topic), 'rb') as f:
        image_tokens.append(json.load(f))

In [5]:
# 시 키워드 토크나이징 데이터 로드
with open('../data/poem_token.pkl', 'rb') as f:
    poem_token = pickle.load(f)

### 2-1. 테스트 코드

In [6]:
# 테스트 코드
with open('../data/beach_token.json', 'rb') as f:
    beach_keyword_token = json.load(f)

In [7]:
# 데이터 모양 확인
beach_keyword_token[0]

{'label': 'beach',
 'img_name': '1',
 'keyword': [['바닷가/NNG_'],
  ['모래/NNG_'],
  ['바다/NNG_'],
  ['물/NNG_', '이/JKS_', '몸/NNG_'],
  ['여름/NNG_'],
  ['대양/NNG_'],
  ['해/NNG_'],
  ['재미/NNG_'],
  ['분명히/MAG_'],
  ['레저/NNG_', '(/SS_', '시간/NNG_', '꺼/VV_', '짐/NNG_', ')/SS_'],
  ['여행/NNG_'],
  ['휴가/NNG_'],
  ['사랑/NNG_'],
  ['아이/NNG_'],
  ['밀리/VV_', '어/EC_', '오/VX_', '는/ETM_', '파도/NNG_'],
  ['자연/NNG_'],
  ['연안/NNG_'],
  ['천국/NNG_'],
  ['자유/NNG_', '(/SS_', '상태/NNP_', ')/SS_'],
  ['남성/NNG_'],
  ['ᄉ/NNG_', 'ᅩᆨ초해수ᄋ/SH_', 'ᅭᆨ자/NNP_', 'ᆼ/SH_'],
  ['ᄀ/NNG_', 'ᅡ족/SH_', '여해/NNP_', 'ᆼ/SH_'],
  ['ᄀ/NNG_', 'ᅧ울/SH_', '바ᄃ/NNP_', 'ᅡ/SH_'],
  ['ᄉ/NNG_', 'ᅩᆨ초해수ᄋ/SH_', 'ᅭᆨ자/NNP_', 'ᆼ/SH_'],
  ['ᄀ/NNG_', 'ᅡ족/SH_', '여해/NNP_', 'ᆼ/SH_'],
  ['ᄀ/NNG_', 'ᅧ울/SH_', '바ᄃ/NNP_', 'ᅡ/SH_'],
  ['ᄉ/NNG_', 'ᅩᆨ초해수ᄋ/SH_', 'ᅭᆨ자/NNP_', 'ᆼ/SH_'],
  ['ᄀ/NNG_', 'ᅡ족/SH_', '여해/NNP_', 'ᆼ/SH_'],
  ['ᄀ/NNG_', 'ᅧ울/SH_', '바ᄃ/NNP_', 'ᅡ/SH_'],
  ['ᄇ/NNG_',
   'ᅡ/SH_',
   'ᄃ/SL_',
   'ᅡᄂ/SH_',
   'ᅳ/NNP_',
   'ᆫ/SH_'

### 3. 데이터에서 명사인 키워드만 추출

In [8]:
def del_none(x):
    d = []
    for j in tqdm(x):
        c = []
        for i in j:
            if i == None:
                pass
            else:
                c.append(i)
        d.append(c)
    return d

In [9]:
def select_noun_va(x):
    new_a = []
    for j in tqdm(x):
        aa = []
        for i in j:   
            if 'NNG' in i:
                aa.append(i)
        new_a.append(aa)
    return new_a

In [34]:
# 명사/NN => 명사
def only_words(x):
    a = []
    for i in tqdm(x):
        b = []
        for j in i:
            b.append(j.split('/')[0])
        a.append(b)
    return a

### 3-1. 이미지

In [10]:
def get_noun_keyword(img_data):
    key_tok = []
    for i in range(len(img_data)):
        key_tok.append(img_data[i]['keyword'])
    flatten_key_tok = []
    for i in key_tok:
        flatten_key_tok.append([y for x in i for y in x])
    noun_kwd = select_noun_va(del_none(flatten_key_tok))
    for i in tqdm(range(len(img_data))):
        img_data[i]['keyword'] = 0
        img_data[i]['keyword'] = list(set(noun_kwd[i]))
    return img_data 

In [11]:
for image_token in image_tokens:
    image_token = get_noun_keyword(image_token)

100%|███████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 83658.53it/s]


### 3-1-1. 테스트 코드

In [12]:
beach_keyword_token = get_noun_keyword(beach_keyword_token)

100%|███████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 83546.88it/s]


In [13]:
beach_keyword_token[0]

{'label': 'beach',
 'img_name': '1',
 'keyword': ['휴가/NNG_',
  '해/NNG_',
  'ᄀ/NNG_',
  '천국/NNG_',
  '시간/NNG_',
  '자유/NNG_',
  '아이/NNG_',
  '짐/NNG_',
  '사랑/NNG_',
  'ᄇ/NNG_',
  '레저/NNG_',
  '대양/NNG_',
  '여름/NNG_',
  'ᄉ/NNG_',
  '파도/NNG_',
  '몸/NNG_',
  '여행/NNG_',
  '연안/NNG_',
  '바다/NNG_',
  '바닷가/NNG_',
  '자연/NNG_',
  '물/NNG_',
  '남성/NNG_',
  '모래/NNG_',
  '재미/NNG_']}

### 3-2. 시

명사 토큰이 없을 때 해당 시를 버리겠다는 의미로 'ㄱ'을 넣어준다.

In [29]:
def get_noun_poem(poem_data):
    flatten_poem_tok = []
    for i in poem_data:
        flatten_poem_tok.append([y for x in i for y in x])
    noun_poem = select_noun_va(del_none(flatten_poem_tok))
    # 시의 경우, 명사 토큰이 없는 경우, 임의로 'ㄱ'을 넣어준다. 해당 시를 버리겠다는 것
    for i in range(len(noun_poem)):
        if len(noun_poem[i]) == 0:
            noun_poem[i] = ['ㄱ']       
    return noun_poem 

In [30]:
noun_poem = get_noun_poem(poem_token)

100%|█████████████████████████████████████████████████████████████████████████| 74554/74554 [00:02<00:00, 34817.55it/s]


In [31]:
type(noun_poem)

list

In [33]:
noun_poem[0]

[['성실/NNG_',
  '근면/NNG_',
  '도/NNG_',
  '로/NNG_',
  '정직/NNG_',
  '삶/NNG_',
  '사랑/NNG_',
  '때/NNG_',
  '때/NNG_',
  '답/NNG_',
  '때/NNG_',
  '작은사람/NNG_',
  '사랑/NNG_',
  '안/NNG_',
  '키/NNG_',
  '사람/NNG_',
  '숲/NNG_']]

### 4. fasttext 모델로 word vector 생성

In [18]:
model = fasttext.load_model('../model/wiki.ko/wiki.ko.bin')

In [19]:
model.get_dimension()

300

### 4-1. 이미지

- 이미지 각 키워드의 명사 토큰을 300차원으로 임베딩한다.
- 이미지를 대표하는 하나의 벡터로 만든다

In [20]:
def embedding_img(image_token):
    tot = []
    for i in tqdm(range(len(image_token))):
        sen = []
        for j in range(len(image_token[i]['keyword'])):
            sen.append(model.get_word_vector(image_token[i]['keyword'][j]))
        tot.append(sen)
    return tot

In [21]:
embeded_vector_image_keyword = []

In [22]:
for image_token in tqdm(image_tokens):
    tot = embedding_img(image_token)
    new_tot = np.array([np.array(i) for i in tot])
    new_tot_samplesum = [ np.sum(i, axis=0) for i in new_tot]
    vector_keyword = np.array(new_tot_samplesum)
    embeded_vector_image_keyword.append(vector_keyword)

 23%|█████████████████▋                                                            | 226/1000 [00:00<00:01, 448.80it/s]


 14%|██████████▋                                                                   | 137/1000 [00:00<00:01, 689.19it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:16<00:00,  1.85s/it]


In [23]:
# 300차원 확인
embeded_vector_image_keyword[0][1].shape

(300,)

In [ ]:
# [embeded_vector_keyword]

### 4-2. 시

- 각 단어를 300차원으로 임베딩한다.
- 시를 대표하는 하나의 벡터로 만든다 (시 단위로 임베딩한 벡터들을 더함)

In [24]:
def embedding_poem(noun_poem):
    total = []
    for i in tqdm(range(len(noun_poem))):
        sen = []
        for j in range(len(noun_poem[i])):
            sen.append(model.get_word_vector(noun_poem[i][j]))
        total.append(sen)
    new_total = np.array([np.array(i) for i in total])
    new_total_samplesum = [ np.sum(i, axis=0) for i in new_total]
    new_total_samplesum = np.array(new_total_samplesum)
    return new_total_samplesum

In [25]:
embeded_vector_poem = embedding_poem(noun_poem)

  0%|                                                                                        | 0/74554 [00:00<?, ?it/s]


TypeError: getWordVector(): incompatible function arguments. The following argument types are supported:
    1. (self: fasttext_pybind.fasttext, arg0: fasttext_pybind.Vector, arg1: str) -> None

Invoked with: <fasttext_pybind.fasttext object at 0x00000282342381B8>, <fasttext_pybind.Vector object at 0x000002823421DC00>, ['성실/NNG_', '하/XSA_', 'ㅁ/ETN_', '이나/JC_', '근면/NNG_', '하/XSA_', 'ㅁ/ETN_', '도/NNG_', '글/MAG_', '쎄/IC_']

In [ ]:
# 300차원 확인
embeded_vector_poem[1].shape

In [ ]:
# embeded_vector_poem 

### 5. 코사인 유사도 구하기

- keyword_vector로 이미지와 가장 걸맞는(a.k.a 가장 유사도가 높은) poem_vector를 구하고, 그 시의 index를 append

In [ ]:
def cosine_similarity(x, y):
    return np.dot(x, y) / (np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y)))

In [ ]:
def return_index(vector_keyword, vector_poem):
    scores = []
    for i in range(len(vector_poem)):
        score = cosine_similarity(vector_keyword, vector_poem[i])
        scores.append(score)
    index = scores.index(max(scores))
    return index

In [ ]:
matched_list = []

In [ ]:
for embeded_vector_keyword in embeded_vector_image_keyword:
    index_list = []
    for i in range(len(embeded_vector_keyword)):
        index = return_index(embeded_vector_keyword[i], embeded_vector_poem)
        index_list.append(index)
    matched_list.append(index_list)

In [ ]:
len(matched_list[0])

In [ ]:
len(set(matched_list[0]))  # 매칭된 시 개수

### 6. 모델을 위한 json 저장

In [ ]:
# replace를 하기 위함
matched_df = pd.DataFrame(matched_list)

In [ ]:
# matched_df shape
# (10, 1000)

In [ ]:
# 유니크한 index list를 구하기 위해 이중 리스트 flatten
unique_index_list = list(set([y for x in matched_list for y in x]))

In [ ]:
for unique_index in unique_index_list:
    matched_df.replace(unique_index, poem_token[unique_index], inplace=True)

In [ ]:
for i in range(len(image_token)):
    matched_poem = matched_df.iloc[i].tolist() #i번째 image topic에 대해 매칭된 시
    for poem_idx in range(len(matched_poem)):
        image_tokens[i][poem_idx]['text'] = matched_poem[poem_idx]
    with open('../data/final/{topic}.json'.format(topic=image_token[i]), 'w', encoding='utf-8') as f:
        json.dump(image_tokens[i], f)